In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('/kaggle/input/delhi-house-price-prediction/MagicBricks.csv')

In [ ]:
df['Per_Sqft'].fillna((df['Price']/df['Area']),inplace=True)
df['Bathroom'].fillna(df['Bathroom'].mode()[0],inplace=True)
df['Furnishing'].fillna(df['Furnishing'].mode()[0],inplace=True)
df['Parking'].fillna(df['Parking'].mode()[0],inplace=True)
df['Type'].fillna(df['Type'].mode()[0],inplace=True)


df[['Parking','Bathroom']]=df[['Parking','Bathroom']].astype('int64')


def grp_loc(locality):
    locality=locality.lower()
    if 'rohini' in locality:
        return 'Rohini Sector'
    elif 'dwarka' in locality:
        return 'Dwarka Sector'
    elif 'shahdara' in locality:
        return 'Shahdara'
    elif 'vasant' in locality:
        return 'Vasant Kunj'
    elif 'paschim' in locality:
        return 'Paschim Vihar'
    elif 'alaknanda' in locality:
        return 'Alaknanda'
    elif 'vasundhara' in locality:
        return 'Vasundhara Enclave'
    elif 'punjabi' in locality:
        return 'Punjabi Bagh'
    elif 'kalkaji' in locality:
        return 'Kalkaji'
    elif 'lajpat' in locality:
        return 'Lajpat Nagar'
    elif 'laxmi' in locality:
        return 'Laxmi Nagar'
    elif 'patel' in locality:
        return 'Patel Nagar'
    else:
        return 'Other'
    
    
df['Locality']=df['Locality'].apply(grp_loc)    
from scipy import stats
z = np.abs(stats.zscore(df[df.dtypes[df.dtypes != 'object'].index]))
df = df[(z < 3).all(axis=1)]

## Date Preprocessing

#### Label Encoding

In [44]:
from sklearn.preprocessing import LabelEncoder

In [45]:
le=LabelEncoder()

In [47]:
col=['Furnishing','Locality','Status','Transaction','Type']

In [48]:
for i in col:
    le.fit(df[i])
    df[i]=le.transform(df[i])
    print(i,df[i].unique())

Furnishing [1 0 2]
Locality [ 9  3  1  5  4  7 11 10  6  0 12  8  2]
Status [1 0]
Transaction [0 1]
Type [1 0]


#### Normalizing the continuous features

In [49]:
from sklearn.preprocessing import MinMaxScaler
min_max=MinMaxScaler()
df[['Area','Price','Per_Sqft']]=min_max.fit_transform(df[['Area','Price','Per_Sqft']])